In [1]:
# import libraries
import numpy as np
import cv2 as cv

In [2]:
# read in single test image
img = cv.imread("/home/leo/Documents/UNI/ICDS/Groupwork/Group1/data/test/normal/IM-0115-0001.jpeg", cv.IMREAD_GRAYSCALE)
print(img.shape)

(1858, 2090)


In [3]:
class conv_layer:
    """
    Convolution layer expects an input dimension (in_dim) of shape (h, w, d), where h and w
    are hight and width of images and d the number of dimensions. 

    Currently there is no padding or stride configuration. The layer operates with no paddling 
    and a stride of one.

    in_dim -> tuple of shape (h,w,d) -> input image dimensions 
    conv_size -> tuple of shape (h, w) -> size of convolution kernel
    kernel_num -> int -> number of kernels
    """
    def __init__(self, in_dim, conv_size=(3,3), kernel_num=4, debug = False):
        self.kernel_num = kernel_num
        self.conv_size = conv_size
        self.conv_kernels = [None] * self.kernel_num

        for i in range(0, self.kernel_num):
            if debug:
                self.conv_kernels[i] = self.debug_conv((conv_size[0], conv_size[1], in_dim[2]))
            else:
                self.conv_kernels[i] = np.random.uniform(-1,1,(conv_size[0], conv_size[1], in_dim[2]))

        self.in_dim = in_dim
        self.out_dim = (self.in_dim[0] - (conv_size[0] - 1), self.in_dim[1] - (self.conv_size[1] - 1), self.kernel_num)
    
    '''
    perform a forward convolution on the specified image.

    img -> np.array of shape (h,w,d)
    
    ret:
    out_img -> np.array of shape (h,w,d)
    '''
    def forward(self, img):
        out_img = np.zeros(self.out_dim)
        for k in range(0, self.kernel_num):
            for h in range(0, self.out_dim[0]):
                for w in range(0, self.out_dim[1]):
                    out_img[h, w, k] = np.sum(img[h:h+self.conv_size[0], w:w+self.conv_size[1],:] * self.conv_kernels[k])
        return out_img
    
    '''
    get out put dimension of this network layer

    ret:
    out_dim -> tuple of shape (h,w,d)
    '''
    def get_out_dim(self):
        return self.out_dim
    
    def debug_conv(self, size):
        kernel = np.zeros(size)
        for i in range(0, size[0],2):
            kernel[i,:,:] = 1
        return kernel


In [12]:
# test conv layer
a = np.zeros((6,6,2))
for w in range(0,a.shape[0], 2):
    for h in range(0,a.shape[1]):
        a[w, h, 0] = 1

for w in range(0,a.shape[0]):
    for h in range(0,a.shape[1], 2):
        a[w, h, 1] = 1

print("input before conv")
print(a[:,:,0])
print(a[:,:,1])
c = conv_layer(in_dim=a.shape, conv_size=(3,3), kernel_num=3)
print("expected out dim")
print(c.get_out_dim())
d = c.forward(a)
print("output conv shape")
print(d.shape)
print("output conv")
print(d[:,:,1])


input before conv
[[1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0.]]
[[1. 0. 1. 0. 1. 0.]
 [1. 0. 1. 0. 1. 0.]
 [1. 0. 1. 0. 1. 0.]
 [1. 0. 1. 0. 1. 0.]
 [1. 0. 1. 0. 1. 0.]
 [1. 0. 1. 0. 1. 0.]]
expected out dim
(4, 4, 3)
output conv shape
(4, 4, 3)
output conv
[[0.46422625 0.51825788 0.46422625 0.51825788]
 [0.26437059 0.31840223 0.26437059 0.31840223]
 [0.46422625 0.51825788 0.46422625 0.51825788]
 [0.26437059 0.31840223 0.26437059 0.31840223]]


In [34]:
class max_pooling_layer:
    """
    Max pooling layer expects an input dimension (in_dim) of shape (h, w, d), where h and w
    are hight and width of images and d the number of dimensions. 
    
    Currently there is no padding or stride configuration. The layer operates with no paddling 
    and a stride of one.

    in_dim -> tuple of shape (h,w,d) -> input image dimensions 
    pooling_size -> tuple of shape (h, w) -> size of pooling filter
    """
    def __init__(self, in_dim, pooling_size=(3,3)):
        self.pooling_size = pooling_size
        self.in_dim = in_dim
        h_overflow = 1 if self.in_dim[0] % self.pooling_size[0] > 0 else 0
        w_overflow = 1 if self.in_dim[1] % self.pooling_size[1] > 0 else 0
        self.out_dim = (int(self.in_dim[0] / self.pooling_size[0]) + h_overflow, int(self.in_dim[0] / self.pooling_size[0]) + w_overflow, self.in_dim[2])
        
    '''
    perform forward pooling on the specified image.

    img -> np.array of shape (h,w,d)
    
    ret:
    out_img -> np.array of shape (h,w,d)
    '''
    def forward(self, img):
        out_img = np.empty(self.out_dim)
        h_overflow = True if self.in_dim[0] / self.pooling_size[0] - self.out_dim[0] > 0 else False
        w_overflow = True if self.in_dim[1] / self.pooling_size[1] - self.out_dim[1] > 0 else False
        for d in range(0, self.out_dim[2]):
            for w in range(0, self.out_dim[0]):
                for h in range(0, self.out_dim[1]):
                    pool_size_h = self.pooling_size[0]
                    pool_size_w = self.pooling_size[1]
                    if h_overflow and h == (self.out_dim[0]-1):
                        pool_size_h = self.in_dim[0] % self.pooling_size[0]
                    if h_overflow and h == (self.out_dim[0]-1):
                        pool_size_w = self.in_dim[1] % self.pooling_size[1]

                    out_img[h,w,d] = np.max(img[h*pool_size_h:h*pool_size_h+pool_size_h, w*pool_size_w:w*pool_size_w+pool_size_w,d])
        return out_img
    '''
    get out put dimension of this network layer

    ret:
    out_dim -> tuple of shape (h,w,d)
    '''
    def get_out_dim(self):
        return self.out_dim

In [37]:
# test pool layer
a = np.zeros((7, 7, 3))
index = 0
for w in range(0,a.shape[0]):
    for h in range(0,a.shape[1]):
        a[w,h,:] = index
        index += 1
print("input before pool")
print(a[:,:,0])
c = max_pooling_layer(in_dim=a.shape, pooling_size=(3,3))
d = c.forward(a)
print("output pool")
print(d[:,:,0])

input before pool
[[ 0.  1.  2.  3.  4.  5.  6.]
 [ 7.  8.  9. 10. 11. 12. 13.]
 [14. 15. 16. 17. 18. 19. 20.]
 [21. 22. 23. 24. 25. 26. 27.]
 [28. 29. 30. 31. 32. 33. 34.]
 [35. 36. 37. 38. 39. 40. 41.]
 [42. 43. 44. 45. 46. 47. 48.]]
output pool
[[16. 19. 20.]
 [37. 40. 41.]
 [44. 47. 48.]]


In [7]:
# currently only ReLU activation function hardcoded
class activation_layer:
    '''
    ReLU activation layer performs a ReLU function on each element in the input image. 
    This means it basically set everything to zero that is smaller then zero.

    in_dim -> tuple of shape (h,w,d) -> input image dimensions 
    '''
    def __init__(self, in_dim,):
        self.in_dim = in_dim

    def forward(self, img):
        out_img = np.stack(np.vectorize(self.relu)(img), axis=0)
        return out_img

    def relu(self, el):
        return(np.maximum(0, el))

In [8]:
# test activation layer
a = np.zeros((2, 3, 3))
index = -4
for k in range(0,a.shape[1]):
    for h in range(0,a.shape[2]):
        a[:, k, h] = index
        index += 1
print("input before activation")
print(a)
r = activation_layer(a.shape)
b = r.forward(a)
print(b)


input before activation
[[[-4. -3. -2.]
  [-1.  0.  1.]
  [ 2.  3.  4.]]

 [[-4. -3. -2.]
  [-1.  0.  1.]
  [ 2.  3.  4.]]]
[[[0. 0. 0.]
  [0. 0. 1.]
  [2. 3. 4.]]

 [[0. 0. 0.]
  [0. 0. 1.]
  [2. 3. 4.]]]


In [9]:
class fully_connected_layer:
    '''
    The fully connected layer transforms the input into a fully connected network
    with an output vector of out_dim.

    in_dim -> tuple of shape (h,w,d) -> input image dimensions 
    out_dim -> int -> defines the number of output nodes
    '''
    def __init__(self, in_dim, out_dim):
        self.in_dim = in_dim
        self.out_dim = out_dim
        w_dim = 1
        for d in in_dim:
            w_dim = w_dim * d
        self.weights = np.ones((self.out_dim, w_dim))
    
    
    def forward(self, img):
        out_vec = np.zeros(self.out_dim)
        img_vec = img.flatten()

        for i in range(0, self.out_dim):
            out_vec[i] = np.sum(img_vec * self.weights[i])
        return out_vec

In [10]:
# test fully connected layer
a = np.ones((2, 3, 3))

print("input before fully connected")
print(a)
c = fully_connected_layer(in_dim=a.shape, out_dim=3)
d = c.forward(a)
print("output pool")
print(d)

input before fully connected
[[[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]]
output pool
[18. 18. 18.]


In [11]:
# test them in succession

img = np.ones((6, 6, 1))
index = 0
for k in range(0,img.shape[0]):
    for h in range(0,img.shape[1]):
        img[k, h] = index
        index += 1

c1 = conv_layer(in_dim=img.shape, conv_size=(3,3), kernel_num=2)
a1 = activation_layer(in_dim=c1.get_out_dim())
p1 = max_pooling_layer(in_dim=c1.get_out_dim(), pooling_size=(3,3))
f1 = fully_connected_layer(in_dim=p1.get_out_dim(), out_dim=2)
print("before cnn")
print(img)
k = c1.forward(img)
print("after conv")
print(k)
k = a1.forward(k)
print("after activation")
print(k)
k = p1.forward(k)
print("after pooling")
print(k)
k = f1.forward(k)
print("after fully connected")
print(k)



before cnn
[[[ 0.]
  [ 1.]
  [ 2.]
  [ 3.]
  [ 4.]
  [ 5.]]

 [[ 6.]
  [ 7.]
  [ 8.]
  [ 9.]
  [10.]
  [11.]]

 [[12.]
  [13.]
  [14.]
  [15.]
  [16.]
  [17.]]

 [[18.]
  [19.]
  [20.]
  [21.]
  [22.]
  [23.]]

 [[24.]
  [25.]
  [26.]
  [27.]
  [28.]
  [29.]]

 [[30.]
  [31.]
  [32.]
  [33.]
  [34.]
  [35.]]]
after conv
[[[ -29.73879441   10.04431367]
  [ -33.16698165   12.38302338]
  [ -36.5951689    14.72173309]
  [ -40.02335614   17.06044281]]

 [[ -50.30791788   24.07657195]
  [ -53.73610513   26.41528166]
  [ -57.16429237   28.75399137]
  [ -60.59247962   31.09270108]]

 [[ -70.87704135   38.10883022]
  [ -74.3052286    40.44753993]
  [ -77.73341584   42.78624965]
  [ -81.16160309   45.12495936]]

 [[ -91.44616483   52.1410885 ]
  [ -94.87435207   54.47979821]
  [ -98.30253932   56.81850792]
  [-101.73072656   59.15721764]]]
after activation
[[[ 0.         10.04431367]
  [ 0.         12.38302338]
  [ 0.         14.72173309]
  [ 0.         17.06044281]]

 [[ 0.         24.07657195]